In [1]:
import pyttsx3;
import pandas
import sys, re
import numpy as np
import nltk
import random
import gensim.models.keyedvectors as word2vec
import speech_recognition as sr
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.corpus import stopwords
from pyemd import emd

ModuleNotFoundError: No module named 'nltk'

In [9]:
model = word2vec.KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin", binary=True)

In [10]:
#PREP

# Record Audio
r = sr.Recognizer()

for index, name in enumerate(sr.Microphone.list_microphone_names()):
    print("Microphone with name \"{1}\" found for `Microphone(device_index={0})`".format(index, name))

df = pandas.read_csv('albert-einstein.csv', names = ["person", "page", "id", "quote"])


stop_words = set()
for word in set(stopwords.words('english')):
  stop_words.add(word)
  stop_words.add(word.capitalize())


Microphone with name "Built-in Microphone" found for `Microphone(device_index=0)`
Microphone with name "Built-in Output" found for `Microphone(device_index=1)`
Microphone with name "Soundflower (2ch)" found for `Microphone(device_index=2)`
Microphone with name "Soundflower (64ch)" found for `Microphone(device_index=3)`
Microphone with name "ACE Sound Effects" found for `Microphone(device_index=4)`
Microphone with name "Loopback Audio" found for `Microphone(device_index=5)`


In [ ]:
def speak(input):
    engine = pyttsx3.init()
    voices = engine.getProperty('voices')
    engine.setProperty('voice',"com.apple.speech.synthesis.voice.maged") #changing index changes voices but ony 0 and 1 are working here
    engine.say(input)
    engine.runAndWait()
    engine.stop()
    listen()
speak("what is it")

In [12]:
def speak(input):
    engine = pyttsx3.init()
    voices = engine.getProperty('voices')
    engine.setProperty('voice',"com.apple.speech.synthesis.voice.daniel") #changing index changes voices but ony 0 and 1 are working here
    engine.say(input)
    engine.runAndWait()
    engine.stop()
    listen()
    listen()

def qa(input):
    user_text = input.replace('hey einstein','')
    user_text = user_text.lower().split()
    user_text = [w for w in user_text if w not in stop_words]
    #print (user_text)
    df2 = pandas.DataFrame(columns=['quote','quote_similarity'])
    quotes = df.quote.astype(str)
    for quote in quotes:
        quote_split = quote.lower().split()
        quote_split = [w for w in quote_split if w not in stop_words]
        distance = model.wmdistance(user_text, quote_split)
            #print (distance)
        df2 = df2.append({'quote': quote, 'quote_similarity': distance}, ignore_index=True)
    df2['Rank'] = df2['quote_similarity'].rank(ascending=1)
        
    #df2.sort_values(by=['quote_similarity'])
    answers = df2.sort_values(by=['quote_similarity']).reset_index().quote.astype(str)
    answer_choice = random.randint(0, 5)
    answer = answers[answer_choice]
    speak(answer)
    
def listen():
  
    with sr.Microphone() as source:
        print("Say something!")
        audio = r.record(source, duration = 4)
    try:
        print("You said: " + r.recognize_google(audio))
        user_voice = r.recognize_google(audio).lower()
        qa(user_voice)
    
    except sr.UnknownValueError:
        #print("Google Speech Recognition could not understand audio")
        listen()
    except sr.RequestError as e:
        #print("Could not request results from Google Speech Recognition service; {0}".format(e))
        listen()
 
listen()

Say something!
You said: what is the meaning of life
Say something!
Say something!
You said: what should I do next
Say something!
Say something!
You said: I don't know
Say something!
You said: thank you
Say something!
You said: Aruba
Say something!
Say something!
Say something!
Say something!
Say something!
You said: thank you
Say something!
Say something!
Say something!
Say something!
Say something!
Say something!
You said: what's the meaning of life
Say something!
You said: yes
Say something!
You said: well it was responding to the word gastro
Say something!
You said: what is Buddhism
Say something!
You said: I don't know what pick up that
Say something!
You said: Rico Abreu


KeyboardInterrupt: 